# 1. Library Preparation

## 1.1. Install Required Libraries

To build this project, you need to install several Python libraries:

- **Flask**: for managing HTTP routes and handling client-server requests.
- **Socket**: for establishing TCP/UDP connections for video and message relay.
- **Pickle**: for serializing and deserializing Python objects, especially for sending video frames.
- **Threading**: for handling concurrent tasks, such as listening for incoming connections while processing data.
- **OpenCV-python**: for capturing video from the webcam and displaying it.
- **Requests**: for making HTTP requests to communicate with the Flask server.
- **Struct**: for packing and unpacking data to send over sockets.

Install these libraries by running the following commands:

```bash
pip install -r requirements.txt
```

**Note:** **socket**, **threading**, **pickle**, and **struct** are part of Python’s standard library and don’t require separate installation.
## 1.2. Import Required Libraries
We will import the required libraries in the following code snippet:

In [ ]:
import socket
import threading
import time
import tkinter as tk 
from tkinter.scrolledtext import ScrolledText
from flask import Flask, request, jsonify
import requests


# 2. Server Configuration
## 2.1. Define Server IP Address and Port Number
To connect two clients on different laptops but within the same network, you should use the local IP address of the laptop running the server. Here’s how you can find and use it:

### Find the Server IP Address:

1. On the server laptop (the laptop running `server.py`), open a terminal or command prompt.
2. Run `ipconfig` (on Windows) or `ifconfig` (on macOS/Linux) to find the local IP address (usually in the form `192.168.x.x` or `10.0.x.x`).

### Set the Server IP in this bellow code:

In `server.py`, where you define the server IP, replace `localhost` or `127.0.0.1` with the server laptop's IP address. For example:

```python
VIDEO_SERVER_IP = "192.168.x.x"  # Replace with the actual IP address of the server laptop
SERVER_PORT = 5000         # Or whatever port you configured



In [ ]:
# UDP server setup for video
SERVER_IP = "192.168.2.16"  # Replace with the actual IP address of the server laptop
VIDEO_SERVER_PORT = 9999    # This port is used for real-time video streaming between clients. It handles the actual transmission of video frames (in this case, using UDP).
SERVER_PORT = 5000          # This port is used by the Flask web server to handle HTTP requests from clients: Registering, listing, starting, stopping 

# Global flag to control server loop
running = True  # Set this to False during shutdown to close the server

## 2.2. UDP Server Setup
We will create a UDP server to receive video frames from the client. The server will listen on the IP address and port number defined above.

In [ ]:
video_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # Create a UDP socket
video_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # Allow reusing the same address
video_socket.bind((SERVER_IP, VIDEO_SERVER_PORT)) # Bind the socket to the server IP and port

## 2.3 Server GUI and Logs handling
We will create a simple GUI using Tkinter to display the server status and logs. The GUI will show the server IP address, port number, and a log of client connections and video forwarding.

In [ ]:
# Define Tkinter GUI class
class ServerGUI:
    
    def __init__(self, root):
        self.root = root
        self.root.title("Video Call Server")

        # Display server information
        self.info_label = tk.Label(root, text=f"Server IP: {SERVER_IP} | Video Port: {VIDEO_SERVER_PORT} | Server Port: {SERVER_PORT}")
        self.info_label.pack(pady=5)

        # Scrolled text widget for logs
        self.log_text = ScrolledText(root, wrap=tk.WORD, width=120, height=50)
        self.log_text.pack(pady=5)
        
        # Disable editing for log_text to make it read-only
        self.log_text.config(state=tk.DISABLED)

        # Entry widget for commands
        self.command_entry = tk.Entry(root, width=50)
        self.command_entry.pack(pady=5)
        self.command_entry.bind("<Return>", self.process_command)  # Bind Enter key to command processing
        
        # Show server starting message
        self.log("Server started successfully!")
        self.log("Waiting for connection from other clients!")

        # Handle closing the server gracefully
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def log(self, message):
        """Insert log message into the GUI in read-only mode with a timestamp."""
        if self.log_text.winfo_exists():  # Check if the log_text widget still exists
            # Get the current time for the timestamp
            current_time = time.strftime("[%H:%M:%S]")  # Format the current time as [HH:MM:SS]
            # Prepare the formatted message with a timestamp
            formatted_message = f"{current_time} {message}"
            # Temporarily enable the log_text widget to insert a new message
            self.log_text.config(state=tk.NORMAL)
            self.log_text.insert(tk.END, formatted_message + "\n")
            self.log_text.see(tk.END)  # Auto-scroll to the latest message
            # Set log_text back to read-only mode
            self.log_text.config(state=tk.DISABLED)

    def process_command(self, event):
        """Process user commands entered in the entry field."""
        command = self.command_entry.get()
        self.command_entry.delete(0, tk.END)  # Clear the entry field
        if command == "/shutdown":
            self.shutdown_server()
        else:
            self.log(f"Unknown command: {command}")

    def shutdown_server(self):
        """Handle shutting down the server gracefully."""
        global running
        self.log("Shutting down the server...")
        running = False  # Set running to 'False' to stop the video forwarding loop
        video_socket.close()  # Close the UDP socket
        self.root.after(2000, self.root.destroy)  # Close the Tkinter window after 2 seconds

    def on_closing(self):
        """Handle closing the server gracefully."""
        self.shutdown_server()

# Initialize Tkinter and Server GUI
root = tk.Tk()
server_gui = ServerGUI(root)

# Helper function to log events in the Tkinter GUI
def log_event(message):
    """Log event with a timestamp."""
    server_gui.log(message)  # Display log in GUI

## 2.4. HTTP Server Setup
We will create an HTTP server using Flask to handle client requests for the video stream. 

In [ ]:

# Data structure to store client info and connection state
# Data structure to store connected clients
clients = {} # This will store client addresses, e.g., { 'Client1': (ip, port, connected_status), 'Client2': (ip, port, connected_status) }
connected_pairs = set()  # Store connected client pairs for forwarding

# Initialize Flask app
app = Flask(__name__)

# Flask route to register a client
@app.route('/register', methods=['POST'])
def register_client():
    client_name = request.json.get("name")
    if client_name in clients:
        log_event(f"Failed registration attempt: Client name '{client_name}' already taken")
        return jsonify({"error": "Client name already taken"}), 400
    clients[client_name] = {"connected": False, "address": None}
    log_event(f"Client '{client_name}' registered successfully")
    return jsonify({"message": f"Client '{client_name}' registered successfully"}), 200

# Flask route to list available clients
@app.route('/clients', methods=['GET'])
def get_clients():
    available_clients = [client for client, data in clients.items() if not data["connected"]]
    log_event("Client requested list of available clients")
    return jsonify({"available_clients": available_clients})

# Flask route to start a call
@app.route('/start_call', methods=['POST'])
def start_call():
    caller = request.json.get("caller")
    callee = request.json.get("callee")
    if caller in clients and callee in clients and not clients[caller]["connected"] and not clients[callee]["connected"]:
        clients[caller]["connected"] = True
        clients[callee]["connected"] = True
        connected_pairs.add((caller, callee))
        log_event(f"Call started between {caller} and {callee}")
        return jsonify({"message": f"Call started between {caller} and {callee}"}), 200
    log_event(f"Failed call attempt: Clients {caller} or {callee} unavailable")
    return jsonify({"error": "Clients unavailable"}), 400

# Flask route to end a call
@app.route('/end_call', methods=['POST'])
def end_call():
    caller = request.json.get("caller")
    callee = request.json.get("callee")
    if (caller, callee) in connected_pairs or (callee, caller) in connected_pairs:
        clients[caller]["connected"] = False
        clients[callee]["connected"] = False
        connected_pairs.discard((caller, callee))
        connected_pairs.discard((callee, caller))
        log_event(f"Call ended between {caller} and {callee}")
        return jsonify({"message": "Call ended"}), 200
    log_event(f"Failed end call attempt: Clients {caller} and {callee} not in a call")
    return jsonify({"error": "Clients not in a call"}), 400



## 2.5 Handle Video Forwarding Between Clients

We will define a function to receive video frames from a client and forward them to the corresponding connected client.

First, we will create a separate thread to run the video forwarding server. This server will listen for video frames sent over **UDP** from clients and forward each frame to the paired client (if connected). When a video frame is received, we identify the sender and find the connected client. If a connected client is available, we forward the video frame to them. If no client is connected, the frame will be discarded.


In [ ]:
def video_forwarding_server():
    # Video forwarding function using UDP
    while running:  # Check the running flag to know when to stop
        try:
            # Receive video frame from one client
            frame_data, client_address = video_socket.recvfrom(65536)
            log_event(f"Received video frame from {client_address}")

            # Identify which client sent the data
            sender_name = next((name for name, data in clients.items() if data["address"] == client_address), None)

            if sender_name:
                log_event(f"Identified sender: {sender_name}")
                
                # Find the connected client to forward the video
                # \ meaning line continuation in Python
                recipient_name = next((callee for caller, callee in connected_pairs if caller == sender_name), None) or \
                                 next((caller for caller, callee in connected_pairs if callee == sender_name), None)
                
                if recipient_name and clients[recipient_name]["address"]:
                    # Forward the video frame to the connected client
                    video_socket.sendto(frame_data, clients[recipient_name]["address"])
                    log_event(f"Forwarded video frame from {sender_name} to {recipient_name} at {clients[recipient_name]['address']}")
                else:
                    log_event(f"No connected recipient found for {sender_name}")
        
        except OSError:
            log_event("Socket closed, stopping video forwarding server.")
            break  # Exit loop if the socket is closed

        except Exception as e:
            log_event(f"Error in video forwarding server: {e}")

# 3. Start the Servers

In [ ]:
# Start the Flask server in a thread
def start_flask_server():
    app.run(host=SERVER_IP, port=SERVER_PORT)

# Start Flask server with a small
threading.Thread(target=start_flask_server, daemon=True).start()

# Start video forwarding server
threading.Thread(target=video_forwarding_server, daemon=True).start()

# Start the Tkinter GUI event loop to handle logs and shutdown events
root.mainloop()
